In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


!cp "/content/gdrive/My Drive/corrected_slices_v2.zip" .
!unzip -qq corrected_slices_v2.zip
!rm corrected_slices_v2.zip
data_path = 'corrected_slices_v2'

Mounted at /content/gdrive


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
import joblib

SEQUENCE_LENGTH = 600
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 5e-4
PLOT_SAVE_DIR = 'predicted_vs_actual_plots_v2'


def process_file(file_path):
    df = pd.read_csv(file_path)

    df['time'] = df['time'] - df['time'].iloc[0]
    df['Coolant_temperature'] = df['Coolant_temperature'] - df['Coolant_temperature'].iloc[0]

    df['Momentary Coolant change rate'] = df['Coolant_temperature'].diff().fillna(0)

    features = df[['Vehicle_Speed']]
    target = df['Momentary Coolant change rate']

    features = features.iloc[:SEQUENCE_LENGTH]
    target = target.iloc[:SEQUENCE_LENGTH]

    return features.values, target.values



def pad_and_normalize(data, scaler, sequence_length=SEQUENCE_LENGTH):
    padded_data = pad_sequences(data, maxlen=sequence_length, dtype='float32', padding='post', truncating='post')
    normalized_data = scaler.transform(padded_data.reshape(-1, padded_data.shape[-1])).reshape(padded_data.shape)
    return normalized_data


scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_original = []
y_original = []
X_augmented = []
y_augmented = []

base_folder_path = '/content/'


folder_path = os.path.join(base_folder_path, f'corrected_slices')
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        features, target = process_file(file_path)

        # slices = filename.split('_')
        # is_original_trip = slices[0] == slices[2] and slices[0] == slices[4]

        # if is_original_trip:
        #     X_original.append(features)
        #     y_original.append(target)
        # else:
        #     X_augmented.append(features)
        #     y_augmented.append(target)
        X_augmented.append(features)
        y_augmented.append(target)

# Pad and convert lists to numpy arrays
# X_original = pad_sequences(X_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
# y_original = pad_sequences(y_original, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
X_augmented = pad_sequences(X_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')
y_augmented = pad_sequences(y_augmented, maxlen=SEQUENCE_LENGTH, dtype='float32', padding='post', truncating='post')

num_test = int(0.1 * len(X_augmented))
X_test = X_augmented[:num_test]
y_test = y_augmented[:num_test]
X_train = X_augmented[num_test:]
y_train = y_augmented[num_test:]

scaler_X.fit(X_train.reshape(-1, X_train.shape[-1]))
scaler_y.fit(y_train.reshape(-1, 1))

joblib.dump(scaler_X, 'scaler_X_temp_added.pkl')
joblib.dump(scaler_y, 'scaler_y_temp_added.pkl')

print(len(X_test))
print(len(X_train))

146
1316


In [ ]:
X_train_normalized = pad_and_normalize(X_train, scaler_X)
y_train_normalized = scaler_y.transform(y_train.reshape(-1, 1)).reshape(y_train.shape)
X_test_normalized = pad_and_normalize(X_test, scaler_X)
y_test_normalized = scaler_y.transform(y_test.reshape(-1, 1)).reshape(y_test.shape)

print('X_train_normalized shape:', X_train_normalized.shape)
print('y_train_normalized shape:', y_train_normalized.shape)
print('X_test_normalized shape:', X_test_normalized.shape)
print('y_test_normalized shape:', y_test_normalized.shape)


X_train_normalized shape: (1316, 600, 1)
y_train_normalized shape: (1316, 600)
X_test_normalized shape: (146, 600, 1)
y_test_normalized shape: (146, 600)


In [ ]:
model = Sequential([
    Bidirectional(LSTM(32, return_sequences=True, input_shape=(SEQUENCE_LENGTH, X_train_normalized.shape[-1]))),
    Dropout(0.2),
    Bidirectional(LSTM(32, return_sequences=True)),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mean_squared_error')

model.fit(X_train_normalized, y_train_normalized, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.2)


Epoch 1/50
17/17 [==============================] - 37s 1s/step - loss: 0.0080 - val_loss: 0.0033
Epoch 2/50
17/17 [==============================] - 20s 1s/step - loss: 0.0039 - val_loss: 0.0028
Epoch 3/50
17/17 [==============================] - 21s 1s/step - loss: 0.0034 - val_loss: 0.0027
Epoch 4/50
17/17 [==============================] - 23s 1s/step - loss: 0.0032 - val_loss: 0.0027
Epoch 5/50
17/17 [==============================] - 22s 1s/step - loss: 0.0032 - val_loss: 0.0027
Epoch 6/50
17/17 [==============================] - 26s 2s/step - loss: 0.0031 - val_loss: 0.0027
Epoch 7/50
17/17 [==============================] - 26s 2s/step - loss: 0.0031 - val_loss: 0.0027
Epoch 8/50
17/17 [==============================] - 25s 1s/step - loss: 0.0030 - val_loss: 0.0027
Epoch 9/50
17/17 [==============================] - 21s 1s/step - loss: 0.0030 - val_loss: 0.0027
Epoch 10/50
17/17 [==============================] - 22s 1s/step - loss: 0.0030 - val_loss: 0.0027
Epoch 11/50
17/17 [

In [ ]:
model.save('modelBLSTM_lightoff_temp_just_speed.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.save('/content/drive/My Drive/model_lightoff(just_speed).h5')

In [ ]:
test_loss = model.evaluate(X_test_normalized, y_test_normalized)
print(f"Test Loss: {test_loss}")

y_pred_normalized = model.predict(X_test_normalized)

y_test_inv = scaler_y.inverse_transform(y_test_normalized.reshape(-1, SEQUENCE_LENGTH))
y_pred_inv = scaler_y.inverse_transform(y_pred_normalized.reshape(-1, SEQUENCE_LENGTH))


5/5 [==============================] - 1s 171ms/step - loss: 0.0027
Test Loss: 0.002730116480961442
5/5 [==============================] - 3s 211ms/step


In [ ]:
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.09828418493270874


In [ ]:
print(len(X_test))

146


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

if not os.path.exists(PLOT_SAVE_DIR):
    os.makedirs(PLOT_SAVE_DIR)

for i in range(len(X_test)):
    # Convert momentary to cumulative
    cumulative_actuals = np.cumsum(y_test_inv[i], axis=0)  # assuming y_test_inv is already the inverse transformed data
    cumulative_predictions = np.cumsum(y_pred_inv[i], axis=0)  # assuming y_pred_inv is already the inverse transformed predictions
    # cumulative_actuals = y_test_inv[i]
    # cumulative_predictions = y_pred_inv[i]


    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_actuals, label='Actual Coolant temp')
    plt.plot(cumulative_predictions, label='Predicted Coolant temp')
    plt.title(f'Trip {i + 1}: Actual vs Predicted Coolant')
    plt.xlabel('Time Steps')
    plt.ylabel('Coolant temp')
    plt.legend()

    plot_filename = os.path.join(PLOT_SAVE_DIR, f'trip_{i + 1}_actual_vs_predicted.png')
    plt.savefig(plot_filename)
    plt.close()  # Close the plot to save memory



In [ ]:
!zip -r data_v2.zip predicted_vs_actual_plots_v2